https://docs.langchain.com/oss/python/langchain/multi-agent#choosing-a-pattern

- as of now we can call another agent as a tools
- handoffs are in progress
- workflows can only use langgraph

In [ ]:
"""
Implementation
Below is a minimal example where the main agent is given 
access to a single subagent via a tool definition:

In this pattern:
The main agent invokes call_subagent1 when it decides the task matches the subagent’s description.
The subagent runs independently and returns its result.
The main agent receives the result and continues orchestration.

Customization Points
Subagent name ("subagent1_name"): This is how the main agent refers to the subagent. Since it influences prompting, choose it carefully.
Subagent description ("subagent1_description"): This is what the main agent “knows” about the subagent. It directly shapes how the main agent decides when to call it.
Input to the subagent: You can customize this input to better shape how the subagent interprets tasks. In the example above, we pass the agent-generated query directly.
Output from the subagent: This is the response passed back to the main agent. You can adjust what is returned to control how the main agent interprets results. In the example above, we return the final message text, but you could return additional state or metadata.

"""

from langchain.tools import tool
from langchain.agents import create_agent


subagent1 = create_agent(model="...", tools=[...])

@tool(
    "subagent1_name",
    description="subagent1_description"
)
def call_subagent1(query: str):
    result = subagent1.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    return result["messages"][-1].content

agent = create_agent(model="...", tools=[call_subagent1])

In [ ]:
"""
Control the input to the subagent
There are two main levers to control the input that the main agent passes to a subagent:
Modify the prompt – Adjust the main agent’s prompt or the tool metadata (i.e., sub-agent’s name and description) to better guide when and how it calls the subagent.
Context injection – Add input that isn’t practical to capture in a static prompt (e.g., full message history, prior results, task metadata) by adjusting the tool call to pull from the agent’s state.
"""
from langchain.agents import AgentState
from langchain.tools import tool, ToolRuntime

class CustomState(AgentState):
    example_state_key: str

@tool(
    "subagent1_name",
    description="subagent1_description"
)
def call_subagent1(query: str, runtime: ToolRuntime[None, CustomState]):
    # Apply any logic needed to transform the messages into a suitable input
    subagent_input = some_logic(query, runtime.state["messages"])
    result = subagent1.invoke({
        "messages": subagent_input,
        # You could also pass other state keys here as needed.
        # Make sure to define these in both the main and subagent's
        # state schemas.
        "example_state_key": runtime.state["example_state_key"]
    })
    return result["messages"][-1].content

In [ ]:
"""
Control the output from the subagent
Two common strategies for shaping what the main agent receives back from a subagent:
Modify the prompt – Refine the subagent’s prompt to specify exactly what should be returned.
Useful when outputs are incomplete, too verbose, or missing key details.
A common failure mode is that the subagent performs tool calls or reasoning but does not include the results in its final message. Remind it that the controller (and user) only see the final output, so all relevant info must be included there.
Custom output formatting – adjust or enrich the subagent’s response in code before handing it back to the main agent.
Example: pass specific state keys back to the main agent in addition to the final text.
This requires wrapping the result in a Command (or equivalent structure) so you can merge custom state with the subagent’s response.
"""

from typing import Annotated
from langchain.agents import AgentState
from langchain.tools import InjectedToolCallId
from langgraph.types import Command


@tool(
    "subagent1_name",
    description="subagent1_description"
)
# We need to pass the `tool_call_id` to the sub agent so it can use it to respond with the tool call result
def call_subagent1(
    query: str,
    tool_call_id: Annotated[str, InjectedToolCallId],
# You need to return a `Command` object to include more than just a final tool call
) -> Command:
    result = subagent1.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    return Command(update={
        # This is the example state key we are passing back
        "example_state_key": result["example_state_key"],
        "messages": [
            ToolMessage(
                content=result["messages"][-1].content,
                # We need to include the tool call id so it matches up with the right tool call
                tool_call_id=tool_call_id
            )
        ]
    })
